#### Preprocessing

In [1]:
# !pip install pytorch_lightning==0.8.5

In [2]:
# !pip install transformers

In [3]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl


from transformers import (
    AdamW,
    MT5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to
[nltk_data]     /usr4/cs440/cueda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



MESSAGE FROM RCS: Setting LC_ALL and LANG to locale value 'en_US.utf8'.  If you wish to use a different locale, use the following code to set it to your prefered locale prior to importing click.
###############################
import os
os.environ["LC_ALL"]="new locale"
os.environ["LANG"]=os.environ["LC_ALL"]
###############################



In [4]:
# Set random seed
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

#### Tuning and Parameters

In [5]:
class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(T5FineTuner, self).__init__()
    self.hparams = hparams
    
    self.model = MT5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
    self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)
  
  def is_logger(self):
    return self.trainer.proc_rank <= 0
  
  def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
    )

  def _step(self, batch):
    lm_labels = batch["target_ids"]
    lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        labels=lm_labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]

    return loss

  def training_step(self, batch, batch_idx):
    loss = self._step(batch)

    tensorboard_logs = {"train_loss": loss}
    return {"loss": loss, "log": tensorboard_logs}
  
  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    tensorboard_logs = {"avg_train_loss": avg_train_loss}
    return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def validation_step(self, batch, batch_idx):
    loss = self._step(batch)
    return {"val_loss": loss}
  
  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    tensorboard_logs = {"val_loss": avg_loss}
    return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]
  
  def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, second_order_closure=None, using_native_amp=None):
    if self.trainer.use_tpu:
      xm.optimizer_step(optimizer)
    else:
      optimizer.step()
    optimizer.zero_grad()
    self.lr_scheduler.step()
  
  def get_tqdm_dict(self):
    tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

    return tqdm_dict

  def train_dataloader(self):
    train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
    dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams)
    return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [6]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log results
      for key in sorted(metrics):
        if key not in ["log", "progress_bar"]:
          logger.info("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

In [7]:
args_dict = dict(
    data_dir="", # path for data files
    output_dir="", # path to save the checkpoints
    model_name_or_path='google/mt5-base',
    tokenizer_name_or_path='t5-base',
    max_seq_length=512,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=2,
    eval_batch_size=2,
    num_train_epochs=2,
    gradient_accumulation_steps=16,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)

#### Loading Data

In [8]:
# from google.colab import drive
# drive.mount('/content/drive')
# from os.path import join as pjoin
# base_dir = 'drive/MyDrive/Colab Notebooks/CS505-Final/'

In [9]:
data_dir = '../data/2017_English_final/GOLD/Subtask_A/'

train_files = []
val_files = []
test_files = []

for root, dirs, files in os.walk(data_dir):
    for file_name in files:
        if 'train' in file_name and '.txt' in file_name:
            train_files.append(os.path.join(data_dir, file_name))
        if 'dev' in file_name and '.txt' in file_name:
            val_files.append(os.path.join(data_dir, file_name))
        if 'test' in file_name and '.txt' in file_name:
            test_files.append(os.path.join(data_dir, file_name))
        
train_data = []
train_labels = []
val_data = []
val_labels = []

sentiment_to_label = {'positive': 2, 'neutral': 1, 'negative': 0}

for file_path in train_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for l in lines:
            entries = l.split('\t')
            train_data.append(entries[2])
            train_labels.append(sentiment_to_label[entries[1]])
            
    
for file_path in val_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for l in lines:
            entries = l.split('\t')
            val_data.append(entries[2])
            val_labels.append(sentiment_to_label[entries[1]])
            

train_data = np.array(train_data)
train_labels = np.array(train_labels)
val_data = np.array(val_data)
val_labels = np.array(val_labels)

In [10]:
train_pos = []
train_neu = []
train_neg = []

for i in range(len(train_data)):
    if (train_labels[i] == 0):
        train_neg.append(train_data[i])
    elif (train_labels[i] == 1):
        train_neu.append(train_data[i])
    elif (train_labels[i] == 2):
        train_pos.append(train_data[i])
        
val_pos = []
val_neu = []
val_neg = []

for i in range(len(val_data)):
    if (val_labels[i] == 0):
        val_neg.append(val_data[i])
    elif (val_labels[i] == 1):
        val_neu.append(val_data[i])
    elif (val_labels[i] == 2):
        val_pos.append(val_data[i])

In [11]:
(len(train_pos), len(train_neu), len(train_neg))

(6904, 6882, 2387)

In [12]:
(len(val_pos), len(val_neu), len(val_neg))

(2412, 2185, 1056)

#### Tokenizing

In [13]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

In [14]:
ids_neg = tokenizer.encode('negative </s>')
ids_neu = tokenizer.encode('neutral </s>')
ids_pos = tokenizer.encode('positive </s>')
len(ids_neg), len(ids_pos), len(ids_neu)

/share/pkg.7/transformers/4.5.0/install/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:174: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


(2, 2, 2)

In [15]:
class TweetDataset(Dataset):
  def __init__(self, tokenizer, pos, neu, neg, max_len=512):
    self.pos_files = pos
    self.neu_files = neu
    self.neg_files = neg
    
    self.max_len = max_len
    self.tokenizer = tokenizer
    self.inputs = []
    self.targets = []

    self._build()
  
  def __len__(self):
    return len(self.inputs)
  
  def __getitem__(self, index):
    source_ids = self.inputs[index]["input_ids"].squeeze()
    target_ids = self.targets[index]["input_ids"].squeeze()

    src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
    target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

    return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}
  
  def _build(self):
    self._buil_examples_from_files(self.pos_files, 'positive')
    self._buil_examples_from_files(self.neu_files, 'neutral')
    self._buil_examples_from_files(self.neg_files, 'negative')
  
  def _buil_examples_from_files(self, lines, sentiment):
    REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

    for text in lines:
      
      line = text.strip()
      line = REPLACE_NO_SPACE.sub("", line) 
      line = REPLACE_WITH_SPACE.sub("", line)
      line = line + ' </s>'

      target = sentiment + " </s>"

       # tokenize inputs
      tokenized_inputs = self.tokenizer.batch_encode_plus(
          [line], max_length=self.max_len, padding='max_length', return_tensors="pt"
      )
       # tokenize targets
      tokenized_targets = self.tokenizer.batch_encode_plus(
          [target], max_length=2, padding='max_length', return_tensors="pt"
      )

      self.inputs.append(tokenized_inputs)
      self.targets.append(tokenized_targets)

In [16]:
def get_dataset(tokenizer, type_path, args):
    if type_path == "val":
        return TweetDataset(tokenizer, val_pos, val_neu, val_neg, max_len=512)
    if type_path == "train":
        return TweetDataset(tokenizer, train_pos, train_neu, train_neg, max_len=512)

#### Train

In [17]:
args_dict.update({'output_dir': 'mt5_imdb_sentiment', 'num_train_epochs':2})
args = argparse.Namespace(**args_dict)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filepath=args.output_dir, prefix="checkpoint", monitor="val_loss", mode="min", save_top_k=5
)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    early_stop_callback=False,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    callbacks=[LoggingCallback()],
)

In [18]:
args

Namespace(adam_epsilon=1e-08, data_dir='', early_stop_callback=False, eval_batch_size=2, fp_16=False, gradient_accumulation_steps=16, learning_rate=0.0003, max_grad_norm=1.0, max_seq_length=512, model_name_or_path='google/mt5-base', n_gpu=1, num_train_epochs=2, opt_level='O1', output_dir='mt5_imdb_sentiment', seed=42, tokenizer_name_or_path='t5-base', train_batch_size=2, warmup_steps=0, weight_decay=0.0)

In [19]:
#### Create model
model = T5FineTuner(args)

In [20]:
trainer = pl.Trainer(**train_params)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [21]:
trainer.fit(model)


  | Name  | Type                        | Params
------------------------------------------------------
0 | model | MT5ForConditionalGeneration | 582 M 


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [22]:
model.model.save_pretrained('mt5_base_tweet_sentiment')

#### Evaluate

In [23]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

In [42]:
# change this line if you need to run on diff plats
test_file_path = '../data/2017_Arabic_train_final/GOLD/SemEval2017-task4-train.subtask-A.arabic.txt'

In [43]:
test_data = []
test_labels = []

with open(test_file_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for l in lines:
        entries = l.split('\t')
        if len(entries) != 3:
            entries = l.split(' ', maxsplit=2)
        test_data.append(entries[2])
        test_labels.append(sentiment_to_label[entries[1]])
            
    
test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [45]:
test_pos = []
test_neu = []
test_neg = []

for i in range(len(test_data)):
    if (test_labels[i] == 0):
        test_neg.append(test_data[i])
    elif (test_labels[i] == 1):
        test_neu.append(test_data[i])
    elif (test_labels[i] == 2):
        test_pos.append(test_data[i])

In [46]:
dataset = TweetDataset(tokenizer, test_pos, test_neu, test_neg,  max_len=512)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [47]:
it = iter(loader)

In [48]:
batch = next(it)
batch["source_ids"].shape

torch.Size([32, 512])

In [50]:
outs = model.model.generate(input_ids=batch['source_ids'].cuda(), 
                              attention_mask=batch['source_mask'].cuda(), 
                              max_length=2)

dec = [tokenizer.decode(ids) for ids in outs]

texts = [tokenizer.decode(ids) for ids in batch['source_ids']]
targets = [tokenizer.decode(ids) for ids in batch['target_ids']]

In [51]:
for i in range(5):
    lines = textwrap.wrap("Tweet:\n%s\n" % texts[i], width=100)
    print("\n".join(lines))
    print("\nActual sentiment: %s" % targets[i])
    print("Predicted sentiment: %s" % dec[i])
    print("=====================================================================\n")

Tweet: RT @AlwaghtNews <unk> <unk> #<unk> _<unk> _<unk> <unk> <unk> #<unk> httpstco62RSkL3JXB</s>
<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
<pad> <pad> <pad> <pad> <pad> <pad> <p

In [52]:


loader = DataLoader(dataset, batch_size=32, num_workers=4)
model.cuda().model.eval()
outputs = []
targets = []
for batch in tqdm(loader):
  outs = model.model.generate(input_ids=batch['source_ids'].cuda(), 
                              attention_mask=batch['source_mask'].cuda(), 
                              max_length=2)

  dec = [tokenizer.decode(ids) for ids in outs]
  target = [tokenizer.decode(ids) for ids in batch["target_ids"]]
  
  outputs.extend(dec)
  targets.extend(target)



  0%|          | 0/105 [00:00<?, ?it/s]

In [53]:
def process_output(prediction):
    return re.sub('<pad> ', '', prediction)

In [54]:
def process_target(target):
    return re.sub('</s>', '', target)

In [55]:
processed_pred = []
for value in outputs:
    processed_pred.append(process_output(value))
    
processed_target = []
for value in targets:
    processed_target.append(process_target(value))

In [56]:
metrics.accuracy_score(processed_target, processed_pred)

0.25007451564828614

In [57]:
n_pos = 0
n_neu = 0
n_neg = 0
for pred in processed_pred:
    if pred == 'positive':
        n_pos+=1
    elif pred == 'neutral':
        n_neu+=1
    elif pred == 'negative':
        n_neg+=1
(n_pos, n_neu, n_neg)

(2839, 516, 0)